In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [2]:
config = {'max_new_tokens': 256, 'temperature': 0.1}

In [4]:
# Calling llama model
llm = CTransformers(model="D:\Code Workspace\DL Model\llama-2-7b-chat.ggmlv3.q8_0.bin",
                    model_type = 'llama',
                    config = config)

d:\Code Workspace\Personal Project\Personal Project - LangChain Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
## Create LLM Chaining
questions_template = "Generate a {selected_topic_level} math quiz on the topic of {selected_topic}. Include {num_quizzes} questions"
questions_prompt = PromptTemplate(input_variables=["selected_topic_level", "selected_topic", "num_quizzes"],
                                    template=questions_template)
questions_chain = LLMChain(llm= llm,
                            prompt = questions_prompt,
                            output_key = "questions")

In [20]:
answer_template = "From this Question:\n {question}\n\n gave me answer to each one of them"
answer_prompt = PromptTemplate(input_variables = ["question"],
                                template = answer_template)
answer_chain = LLMChain(llm = llm,
                        prompt = answer_prompt,
                        output_key = "answer")

In [21]:
## Create Sequential Chaining
seq_chain = SequentialChain(chains = [questions_chain, answer_chain],
                            input_variables = ['selected_topic_level', 'selected_topic', 'num_quizzes'],
                            # Here we return multiple variables
                            output_variables = ['questions', 'answer'])

ValidationError: 1 validation error for SequentialChain
__root__
  Missing required input keys: {'question'}, only had {'selected_topic', 'questions', 'num_quizzes', 'selected_topic_level'} (type=value_error)